# Process pCO2 data from R/V Svea

In [26]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [127]:
import datahandler
import json
linear_regression_log = json.load(open("./linear_regression_log.json"))
datahandler.print_linear_regression_log(linear_regression_log)

Year      Cruise      Number  Period
2020      10_01       1       ['2020-01-10', '2020-01-17']
2021      10_21       1       ['2021-11-08', '2021-11-15']
2021      10_23       1       ['2021-12-06', '2021-12-13']
2021      10_23       2       ['2021-12-14', '2021-12-17']


In [130]:

year = '2021'
id = '10_22'
start = '2021-11-17'
stop = '2021-11-20'

datahandler.modify_linear_regression_log(linear_regression_log, year, id, start, stop, action='add')
datahandler.print_linear_regression_log(linear_regression_log)

Year      Cruise      Number  Period
2020      10_01       1       ['2020-01-10', '2020-01-17']
2021      10_21       1       ['2021-11-08', '2021-11-15']
2021      10_23       1       ['2021-12-06', '2021-12-13']
2021      10_23       2       ['2021-12-14', '2021-12-17']
2021      10_22       1       ['2021-11-17', '2021-11-20']


In [95]:
json.dump(linear_regression_log, open("./linear_regression_log.json", 'w'))

In [99]:
path_co2 = "/CO2FT_A"
path = (
    "./example_data"
    + path_co2
)

from read_data import ProcessData

processed_data = []
for regression_period in linear_regression_log["2021"]["10_23"]['regression_periods']:
    print(regression_period)
    data = ProcessData(path=path, regression_period=regression_period)
    processed_data.append(data)

"""processed_data = []
for cruise_key, regression_periods in linear_regression_log["2021"].items():
    for regression_period in regression_periods["regression_periods"]:
        print(regression_period)
        data = ProcessData(path=path, regression_period=regression_period)
        processed_data.append(data)
"""

['2021-12-06', '2021-12-13']
./example_data/CO2FT_A
95 files in list
add files witin period to list
./example_data/CO2FT_A/CO2FT_A_State_Zero
17 files in list
add files witin period to list
./example_data/CO2FT_A/CO2FT_A_State_Flush
17 files in list
add files witin period to list
./example_data/CO2FT_A/CO2FT_A_Signal_Raw
17 files in list
add files witin period to list
./example_data/CO2FT_A/CO2FT_A_Signal_Ref
17 files in list
add files witin period to list
./example_data/CO2FT_A/CO2FT_A_T_Gas
17 files in list
add files witin period to list
./example_data/CO2FT_A/CO2FT_A_P_In
17 files in list
add files witin period to list
./example_data/CO2FT_A/CO2FT_A_P_NDIR
17 files in list
add files witin period to list
./example_data/CO2FT_A/CO2FT_A_PCO2_Corr
17 files in list
add files witin period to list
./example_data/CO2FT_A/CO2FT_A_Signal_Proc
17 files in list
add files witin period to list
start reading files from 2021-12-06 to 2021-12-13
ZeroCycle
ZeroCycle 48 files read from cruise from 202

'processed_data = []\nfor cruise_key, regression_periods in linear_regression_log["2021"].items():\n    for regression_period in regression_periods["regression_periods"]:\n        print(regression_period)\n        data = ProcessData(path=path, regression_period=regression_period)\n        processed_data.append(data)\n'

In [45]:

for data in processed_data:

    data.process_data()
    # process_data processed measurments as default, to process zerodata, name the data you want to process
    data.process_data(dataframe_name="zerocycles_df")
    data.process_data(dataframe_name="zerocycles_mean_df")
    data.process_data(dataframe_name="zerocycles_usedvalues_df")

selection empty! In cycle no 0.0
selection empty! In cycle no 1.0
selection empty! In cycle no 2.0
selection empty! In cycle no 3.0
selection empty! In cycle no 4.0
selection empty! In cycle no 5.0
selection empty! In cycle no 6.0
selection empty! In cycle no 7.0
selection empty! In cycle no 20.0
selection empty! In cycle no 46.0


In [49]:

import os
path=os.path.abspath("../processed_data")
for data in processed_data:
    data.save_df_data(path=path, df_name="zerocycles_df_processed")
    data.save_df_data(path=path, df_name="measurements_df_processed")
    data.save_df_data(path=path, df_name="zerocycles_mean_df_processed")
    data.save_df_data(path=path, df_name="zerocycles_usedvalues_df_processed")
    data.save_processed_data(path=path)


Q_flag
0: Operate
state
State_Measure
Q_flag
0: Operate
state
State_Measure


#### Zeroing data

In the code below we will look at the zeroing data. Check how zero values look to see if more data should be removed and if and/or if the expedition should be divided into more than one regression.

- Splitting expedition: set new start and stop data and re-run calculations above. One run can hold only one expedition, i.e. one linear regression over zero values from start-stop data without breaks.

In [22]:
import plotting
import datahandler
from bokeh.models import ColumnDataSource
from bokeh.plotting import show
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [46]:

rawdata = processed_data[1]
source = ColumnDataSource(rawdata.zerocycles_df.loc[rawdata.zerocycles_df["State_Zero"]==1])

p = plotting.times_series_scatter_plot(
    data=rawdata.zerocycles_df,
    x="datetime_objects",
    y="two_beam_signal",
    source=source,
    legend_field="Q_flag",
    factors=list(set(source.data["Q_flag"])),
    title="rawdata.zerocycles_df only zero state 1",
)
p.xaxis.axis_label = "date"
p.yaxis.axis_label = "two_beam_signal"
p.legend.title = "zeroing data"

rawdata.zerocycles_mean_df["datetime_objects"] = [
    datahandler.get_datetime_object(float(x))
    for x in rawdata.zerocycles_mean_df["timestamp"]
]

p.circle(
        x=rawdata.zerocycles_mean_df["datetime_objects"],
        y=rawdata.zerocycles_mean_df["two_beam_signal"],
        color="green",
        fill_alpha=0.6,
        legend_label="mean of each zero cycle",
        size=8,
    )

p.circle(source =rawdata.zerocycles_usedvalues_df,
        x="datetime_objects",
        y="two_beam_signal",
        fill_alpha=0,
        color = 'black',
        legend_label="zero values used for mean",
        size=8,
    )


show(p)

['28672: operate switch from zero to flush?', '0: Operate', '12416: operate zeroing startup?']


### Measurement data

Below we look at data from the measurement state

In [48]:

rawdata.measurements_df_processed["datetime_objects"] = [
    datahandler.get_datetime_object(float(x))
    for x in rawdata.measurements_df_processed["timestamp"]
]
#rawdata.measurements_df["Q_flag"] = rawdata.measurements_df["Q_flag"].apply(str)
data=rawdata.get_valid_data(rawdata.measurements_df_processed, validation_dict={'Q_flag': ['0: Operate'], "state": ["State_Measure"]})
data.head()
source = ColumnDataSource(data)
#source = ColumnDataSource(rawdata.measurements_df)

p = plotting.times_series_scatter_plot(
    data=data,
    #data =rawdata.measurements_df,
    x="datetime_objects",
    y="pco2",
    source=source,
    legend_field="Q_flag",
    factors=list(set(source.data["Q_flag"])),
    title="rawdata.measurements_df_processed",
)

p.circle(source =data,
        x="datetime_objects",
        y="PCO2_Corr",
        fill_alpha=0,
        color = 'red',
        legend_label="unprocessed",
        size=8,
    )

p.xaxis.axis_label = "date"
p.yaxis.axis_label = "pCO2 corrected"
p.legend.title = "rawdata.measurements_df_processed"

show(p)

Q_flag
0: Operate
state
State_Measure
['0: Operate']
